# Extracellular E-Field Stimulus

This notebook demonstrates how to apply a spatially uniform extracellular electric field stimulus to neurons in a SONATA network.

The `spatially_uniform_e_field` stimulus allows you to apply electric field components (Ex, Ey, Ez) that can be constant (DC) or oscillating (AC) with configurable frequency and phase. This is useful for modeling transcranial electrical stimulation (TES), deep brain stimulation (DBS), or other extracellular field effects.

## Compiling mechanisms

First, compile the NEURON mechanisms:

In [ ]:
!nrnivmodl ../mechanisms

## Import libraries

In [ ]:
import json
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("white")

from bluecellulab import CircuitSimulation

## Create simulation configuration with e-field stimulus

We'll create a SONATA simulation config that applies a spatially uniform e-field to a target population. The field has:
- DC component: Ex=100 V/m, Ey=-50 V/m, Ez=50 V/m
- AC component: Ex=50 V/m at 10 Hz
- Ramp up/down for smooth transitions

In [ ]:
# Load the base simulation config
base_config_path = Path("./sim_quick_scx_sonata_multicircuit/simulation_config.json")
with open(base_config_path, 'r') as f:
    sim_config = json.load(f)

# Add extracellular e-field stimulus
sim_config["inputs"] = {
    "extracellular_efield": {
        "input_type": "extracellular_stimulation",
        "module": "spatially_uniform_e_field",
        "delay": 50.0,
        "duration": 200.0,
        "node_set": "l4pc",
        "fields": [
            {"Ex": 100, "Ey": -50, "Ez": 50, "frequency": 0},
            {"Ex": 50, "Ey": 0, "Ez": 0, "frequency": 10}
        ],
        "ramp_up_time": 10.0,
        "ramp_down_time": 10.0
    }
}

# Save modified config
modified_config_path = Path("./sim_quick_scx_sonata_multicircuit/simulation_config_efield.json")
with open(modified_config_path, 'w') as f:
    json.dump(sim_config, f, indent=2)

print("E-field stimulus configuration:")
print(json.dumps(sim_config["inputs"]["extracellular_efield"], indent=2))

## Initialize simulation with e-field stimulus

In [ ]:
# Create simulation
sim = CircuitSimulation(modified_config_path)

# Select a few cells from the target population
target_cells = sim.circuit_access.get_target_cell_ids("l4pc")
cells_to_simulate = list(target_cells)[:3]

print(f"Simulating {len(cells_to_simulate)} cells: {cells_to_simulate}")

## Instantiate cells with e-field stimulus

Use the `add_extracellular_stimuli=True` flag to enable e-field application:

In [ ]:
sim.instantiate_gids(
    cells=cells_to_simulate,
    add_extracellular_stimuli=True  # Enable e-field stimuli
)

print(f"Instantiated {len(sim.cells)} cells")
print(f"E-field sources created: {len(sim._efield_sources)}")

## Add voltage recordings

In [ ]:
# Record soma voltage for all cells
for cell_id in sim.cells:
    sim.cells[cell_id].add_voltage_recording()

## Run simulation

In [ ]:
sim.run()
print("Simulation completed!")

## Plot results

Visualize the voltage traces showing the effect of the e-field stimulus:

In [ ]:
fig, axes = plt.subplots(len(cells_to_simulate), 1, figsize=(12, 3*len(cells_to_simulate)), sharex=True)
if len(cells_to_simulate) == 1:
    axes = [axes]

for i, cell_id in enumerate(cells_to_simulate):
    cell = sim.cells[cell_id]
    time = cell.get_time()
    voltage = cell.get_voltage_recording()
    
    axes[i].plot(time, voltage, linewidth=0.8)
    axes[i].set_ylabel(f"Cell {cell_id}\nVoltage (mV)")
    axes[i].axvspan(50, 60, alpha=0.2, color='green', label='Ramp up')
    axes[i].axvspan(60, 240, alpha=0.2, color='yellow', label='E-field active')
    axes[i].axvspan(240, 250, alpha=0.2, color='orange', label='Ramp down')
    axes[i].grid(True, alpha=0.3)
    if i == 0:
        axes[i].legend(loc='upper right')

axes[-1].set_xlabel("Time (ms)")
plt.suptitle("Voltage Response to Extracellular E-Field Stimulus", fontsize=14, y=1.0)
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
1. **Creating** a SONATA config with `spatially_uniform_e_field` stimulus
2. **Configuring** multiple field components (DC + AC)
3. **Applying** the stimulus with ramp up/down
4. **Visualizing** the neuronal response to the e-field

### Key parameters:
- `fields`: List of field components with Ex, Ey, Ez (V/m), frequency (Hz), phase (rad)
- `ramp_up_time`, `ramp_down_time`: Smooth transitions (ms)
- `delay`, `duration`: Timing control (ms)
- `node_set`: Target population

The e-field is applied to all segments of each neuron based on their 3D spatial positions relative to the soma, providing realistic modeling of extracellular stimulation effects.